In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import csv
import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report
from human_ai.code.evaluate import save_predictions
from human_ai.code.dataset import get_train_dataset

dataset = get_train_dataset()
train_set = dataset['train']
test_set = dataset['test']

KeyError: "Column train not in the dataset. Current columns in the dataset: ['id', 'text', 'label']"

In [2]:
# Use logistic regression to predict the class
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(train_set['text'])

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = 'cuda:1'
model_name = "roberta-base-openai-detector" # 0.65525

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.eval()
model.to(device)

2023-03-13 16:51:32.982296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 16:51:34.487629: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-13 16:51:34.487831: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-13 16:51:34.487846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [5]:
train_logits = []
test_logits = []

for sample in tqdm(train_set):
    # Transform input tokens 
    inputs = tokenizer(sample['text'], return_tensors="pt")
    # Model apply
    try:
        outputs = model(**inputs.to(device))
        train_logits.append(outputs.logits.softmax(dim=-1).tolist()[0])
    except:
        train_logits.append([0, 0])
        print("Error")
    
for sample in tqdm(test_set):
    # Transform input tokens 
    inputs = tokenizer(sample['text'], return_tensors="pt")
    # Model apply
    try:
        outputs = model(**inputs.to(device))
        test_logits.append(outputs.logits.softmax(dim=-1).tolist()[0])
    except:
        test_logits.append([0, 0])
        print("Error")

 45%|███████████████████████████████████████████████                                                          | 1525/3400 [00:37<00:44, 42.48it/s]

Error


 47%|█████████████████████████████████████████████████▏                                                       | 1591/3400 [00:39<00:44, 40.57it/s]

Error


 73%|████████████████████████████████████████████████████████████████████████████▍                            | 2476/3400 [01:02<00:21, 43.35it/s]

Error


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:15<00:00, 39.24it/s]


In [28]:
X_train = np.hstack((X.toarray(), np.array(train_logits)))
X_test = np.hstack((vectorizer.transform(test_set['text']).toarray(), np.array(test_logits)))

In [29]:
X_train.shape, X_test.shape

((3400, 10002), (600, 10002))

In [30]:
from sklearn.metrics import accuracy_score

clf = LogisticRegression()
clf.fit(X_train, train_set['label'])
predictions = clf.predict(X_test)
accuracy_score(test_set['label'], predictions)

0.6883333333333334

In [ ]:
# baseline 0.6166666666666667
# adding roberta 0.6883333333333334
# 

In [12]:
for a in dataset:
    print(a)
    break

train


In [9]:
from human_ai.code.dataset import get_train_dataset,get_test_dataset 
import json
from pathlib import Path

mode = 'test'
dataset = get_test_dataset()

root_path = Path('logits')
save_path = root_path / mode
save_path.mkdir(exist_ok=True, parents=True)


logits = {}

for sample in tqdm(dataset):
    text, idx = sample['text'], sample['id']
    # Transform input tokens 
    inputs = tokenizer(sample['text'], return_tensors="pt")
    # Model apply
    try:
        outputs = model(**inputs.to(device))
        logits[idx] = outputs.logits.softmax(dim=-1).tolist()[0]
    except:
        logits[idx] = [0, 0]
        print("Error")

with open(save_path/'logits.json', 'w') as f:
    json.dump(logits, f)

 10%|████████▌                                                                          | 412/4000 [00:10<01:41, 35.28it/s]

Error


 53%|███████████████████████████████████████████▎                                      | 2113/4000 [00:54<00:44, 42.40it/s]

Error


 92%|███████████████████████████████████████████████████████████████████████████▌      | 3688/4000 [01:35<00:08, 35.80it/s]

Error


100%|██████████████████████████████████████████████████████████████████████████████████| 4000/4000 [01:43<00:00, 38.54it/s]


In [ ]:
outputs.